In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('final-features-rgb.csv')
X = df[[f'feature_{i}' for i in range(13)]]
y = df['party']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
# extract model metrics
def extract_metrics(report):
    metrics = {}
    lines = report.split("\n")
    for line in lines:
        if line.startswith(' ') and len(line.split()) > 1:
            parts = line.split()
            if len(parts) >= 5:
                label = parts[0]
                try:
                    precision = float(parts[1])
                    recall = float(parts[2])
                    f1_score = float(parts[3])
                    support = int(parts[4])
                    metrics[label] = {
                        'Precision': precision,
                        'Recall': recall,
                        'F1-Score': f1_score,
                        'Support': support
                    }
                except ValueError:
                    continue  
    return metrics

In [ ]:
# 1. Random Forest 
rf = RandomForestClassifier(class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
report_rf = classification_report(y_test, y_pred_rf)
metrics_rf = extract_metrics(report_rf)
print("=== Random Forest ===")
print(report_rf)

=== Random Forest ===
              precision    recall  f1-score   support

         afd       0.77      0.46      0.58       411
         cdu       0.52      0.68      0.59      1138
         csu       0.54      0.71      0.61      1355
         fdp       0.85      0.77      0.81       841
     gruenen       0.87      0.50      0.63       532
       linke       0.60      0.53      0.56       845
         spd       0.66      0.42      0.52       563

    accuracy                           0.62      5685
   macro avg       0.69      0.58      0.61      5685
weighted avg       0.65      0.62      0.62      5685



In [ ]:
# 2. Support Vector Machine (SVM) 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm = SVC(kernel='rbf', class_weight='balanced', random_state=42)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
report_svm = classification_report(y_test, y_pred_svm)
metrics_svm = extract_metrics(report_svm)
print("=== Support Vector Machine (SVM) ===")
print(report_svm)

=== Support Vector Machine (SVM) ===
              precision    recall  f1-score   support

         afd       0.38      0.64      0.48       411
         cdu       0.48      0.54      0.51      1138
         csu       0.58      0.40      0.47      1355
         fdp       0.74      0.73      0.74       841
     gruenen       0.48      0.54      0.51       532
       linke       0.47      0.40      0.43       845
         spd       0.46      0.54      0.50       563

    accuracy                           0.52      5685
   macro avg       0.51      0.54      0.52      5685
weighted avg       0.53      0.52      0.52      5685



In [ ]:
# 3. MLP Classifier 
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation='relu',
    learning_rate_init=0.001,
    max_iter=1000,
    alpha=0.0001,
    random_state=42
)

mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)
report_mlp = classification_report(y_test, y_pred_mlp)
metrics_mlp = extract_metrics(report_mlp)
print("=== MLP Classifier ===")
print(report_mlp)

=== MLP Classifier ===
              precision    recall  f1-score   support

         afd       0.47      0.45      0.46       411
         cdu       0.49      0.57      0.53      1138
         csu       0.52      0.53      0.53      1355
         fdp       0.75      0.73      0.74       841
     gruenen       0.55      0.52      0.54       532
       linke       0.45      0.43      0.44       845
         spd       0.48      0.40      0.44       563

    accuracy                           0.53      5685
   macro avg       0.53      0.52      0.52      5685
weighted avg       0.53      0.53      0.53      5685



In [ ]:
metrics_all = {}

for label in metrics_rf:
    metrics_all[label] = {
        'Random Forest': metrics_rf[label]['F1-Score'],
        'SVM': metrics_svm[label]['F1-Score'],
        'MLP': metrics_mlp[label]['F1-Score']
    }

metrics_df = pd.DataFrame(metrics_all).T

metrics_df["Average"] = metrics_df.mean(axis=1)

average_row = metrics_df.mean(numeric_only=True)
average_row.name = "Average"
metrics_df = pd.concat([metrics_df, average_row.to_frame().T])

#Output
print("=== Modellvergleich ===")
print(metrics_df.round(2))

=== Modellvergleich ===
         Random Forest   SVM   MLP  Average
afd               0.58  0.48  0.46     0.51
cdu               0.59  0.51  0.53     0.54
csu               0.61  0.47  0.53     0.54
fdp               0.81  0.74  0.74     0.76
gruenen           0.63  0.51  0.54     0.56
linke             0.56  0.43  0.44     0.48
spd               0.52  0.50  0.44     0.49
Average           0.61  0.52  0.53     0.55
